In [1]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 11.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [3]:
model = DiscreteBayesianNetwork([('Prize', 'Host'), ('Choice', 'Host')])

In [4]:
cpd_prize = TabularCPD(variable='Prize', variable_card=3,
                       values=[[1/3], [1/3], [1/3]],
                       state_names={'Prize': [1, 2, 3]})

In [5]:
cpd_choice = TabularCPD(variable='Choice', variable_card=3,
                        values=[[1/3], [1/3], [1/3]],
                        state_names={'Choice': [1, 2, 3]})

In [6]:
host_values = np.zeros((3, 9))

In [7]:
host_doors = [1, 2, 3]
for i, prize in enumerate(host_doors):
    for j, choice in enumerate(host_doors):
        valid = [door for door in host_doors if door != prize and door != choice]
        for v in valid:
            host_values[v-1][3*i + j] = 1 / len(valid)

cpd_host = TabularCPD(variable='Host', variable_card=3,
                      values=host_values,
                      evidence=['Prize', 'Choice'],
                      evidence_card=[3, 3],
                      state_names={'Host': [1, 2, 3], 'Prize': [1, 2, 3], 'Choice': [1, 2, 3]})


In [8]:
model.add_cpds(cpd_prize, cpd_choice, cpd_host)
model.check_model()


True

In [9]:
inference = VariableElimination(model)

In [10]:
query_result = inference.query(variables=['Prize'],
                               evidence={'Choice': 1, 'Host': 3})

In [11]:
print("P(Prize | Choice=1, Host=3):")
print(query_result)

P(Prize | Choice=1, Host=3):
+----------+--------------+
| Prize    |   phi(Prize) |
+==========+==============+
| Prize(1) |       0.3333 |
+----------+--------------+
| Prize(2) |       0.6667 |
+----------+--------------+
| Prize(3) |       0.0000 |
+----------+--------------+


In [12]:
print("\nBest strategy: switch to the door with highest probability.")


Best strategy: switch to the door with highest probability.
